In [ ]:
import os
import json
import datetime
import asyncio
from typing import Optional, List

from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.base import OrTerminationCondition

In [ ]:
# --- LLM Client Configuration ---
config_list_path = "your_api_key.json"
# Example JSON file content:
"""[
    {
        "model": "gpt-4",
        "api_key": "sk-your_api_key"
    }
]"""

try:
    with open(config_list_path, 'r', encoding='utf-8') as f:
        config_data = json.load(f)

    if isinstance(config_data, list) and config_data:
        llm_config = config_data[0]
    elif isinstance(config_data, dict):
        llm_config = config_data
    else:
        raise ValueError(f"No valid configuration found in {config_list_path}.")

  
    model_name = llm_config.get("model")
    if not model_name:
        raise ValueError("Model name ('model') not found in configuration file.")

    api_key = llm_config.get("api_key")
    if not api_key:
        raise ValueError("OpenAI API key ('api_key') not found in configuration file.")

    
    model_client_gpt4 = OpenAIChatCompletionClient(
        model=model_name,  
        api_key=api_key,
    )
    model_client_gpt4o = OpenAIChatCompletionClient(
        model="gpt-4o",     
        api_key=api_key,
    )

except FileNotFoundError:
    print(f"Error: Configuration file not found: {config_list_path}")
    exit(1)
except (json.JSONDecodeError, ValueError, KeyError) as e:
    print(f"Error: Configuration error ({config_list_path}): {e}")
    exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    exit(1)

In [ ]:
# --- Tool Functions ---
async def ask_human_input(prompt: str, cancellation_token: Optional[CancellationToken] = None) -> str:
    print("\n**********************************")
    indented = "\n".join(f"  > {line}" for line in prompt.splitlines())
    print(f"Clarification Agent asks:\n{indented}")
    print("**********************************")
    try:
        return (await asyncio.to_thread(input, "Your input: ")).strip()
    except Exception:
        return "[USER INPUT ERROR]"
    finally:
        print()


def log_event(event_details: str) -> str:
    """Saves the specified event detail string to a log file ('assistant_log.txt') with a timestamp."""
    log_file = "assistant_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] {event_details}\n")
    return f"Event logged successfully to {log_file}: {event_details}"


def add_scheduled_event(subject: str, attendees: List[str], time_preference: str, duration_minutes: int) -> str:
    """Saves a simulated calendar event to 'calendar_log.txt'."""
    log_file = "calendar_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] Scheduled Event (Simulated): Subject='{subject}', Attendees='{', '.join(attendees)}', Time='{time_preference}', Duration='{duration_minutes} minutes'\n")
    return f"Simulated event '{subject}' logged successfully to {log_file}."


def save_email_draft(to: str, subject: str, body: str) -> str:
    """Saves the generated email draft (to, subject, body) to a file ('email_drafts.txt')."""
    draft_file = "email_drafts.txt"
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    draft_content = f"""--- Draft Saved: {timestamp} ---
To: {to}
Subject: {subject}
--- Body ---
{body}
--- End Draft ---
"""
    with open(draft_file, "a", encoding="utf-8") as f:
        f.write(draft_content + "\n")
    return f"Email draft for {to} with subject '{subject}' saved to {draft_file}."


def add_todo_item(task: str) -> str:
    """Adds the given task string to a to-do list file ('todo_list.txt')."""
    todo_file = "todo_list.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    with open(todo_file, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] - {task}\n")
    return f"Task '{task}' added to the To-Do list ({todo_file})."

In [ ]:
# --- Agent Definitions ---
user_proxy = UserProxyAgent(
    name="Mehmet_Gur",
    input_func=ask_human_input,
    description="Represents the end-user Mehmet_Gur. Initiates requests and provides input via the UI/terminal when asked."
)
orchestrator = AssistantAgent(
    name="Orchestrator",
    model_client=model_client_gpt4o,
    system_message="""You are the Orchestrator/Supervisor Agent managing a multi-agent team. Your goal is to fulfill the user's request by following the plan provided by Intent_Task_Router.

Available Agents: Input_Preprocessor, Intent_Task_Router, Research_Agent, Summarization_Agent, Email_Drafting_Agent, Email_Processing_Agent, To_Do_List_Agent, Scheduling_Calendar_Agent, Clarification_Agent, Analytics_Logging_Agent, Mehmet_Gur.

Core Responsibilities:
1.  Receive and understand the plan.
2.  Delegate tasks sequentially to the appropriate agents named in the plan. Clearly state the full name of the agent (e.g., 'Research_Agent, please research...').
3.  Provide necessary inputs, extracting relevant information from previous steps or user clarifications.
4.  If essential information is missing (e.g., manager's email for saving), instruct Clarification_Agent to ask Mehmet_Gur. Wait for the response before proceeding with the step requiring the information.
5.  For the Research_Agent, provide the research topic clearly.
6.  For tool-using agents (Email_Processing_Agent, To_Do_List_Agent, Scheduling_Calendar_Agent, Analytics_Logging_Agent), explicitly instruct them WHEN to call their specific tool and provide the EXACT required arguments.
7.  Simplified Email Workflow:
    a) Instruct Email_Drafting_Agent to draft the email body.
    b) Receive the body from Email_Drafting_Agent.
    c) Instruct Email_Processing_Agent to analyze the sentiment of the received body.
    d) Instruct Email_Processing_Agent to CALL THE TOOL 'save_email_draft' providing the 'to', 'subject', and 'body' received in step (b).
    e) Wait for and verify the confirmation message from the tool call before moving to the next planned step.
8.  Wait for and verify responses or tool confirmations from all agents before proceeding to the next step in the plan.
9.  Handle agent errors reasonably (e.g., retry, ask for clarification, or report inability to complete).
10. Ensure ALL planned steps are completed in sequence. DO NOT skip steps.
11. Adhere strictly to agent roles.

Critical Workflow Points:
*   Delegation: Always clearly name the agent being instructed.
*   Tool Calls: Explicitly instruct agents like Email_Processing_Agent to 'Call the [tool_name] tool with arguments: ...'.
*   Final Output: After all steps are complete, formulate a concise summary prefixed EXACTLY with "FINAL_OUTPUT: ", and end the ENTIRE message IMMEDIATELY with "TERMINATE".
*   Do not send any message after the FINAL_OUTPUT/TERMINATE message.

Think step-by-step based on the plan and the last message before deciding which agent to instruct next.""",
    tools=[],
    reflect_on_tool_use=False,
    description="The main orchestrator managing the workflow and delegating tasks using GPT-4o."
)
input_preprocessor = AssistantAgent(
    name="Input_Preprocessor",
    model_client=model_client_gpt4,
    system_message="""You are the Input Preprocessing Agent.
1.  Receive the raw user query from the Orchestrator.
2.  Clean grammar, spelling, and identify key entities.
3.  Your ONLY output MUST be the cleaned query itself, prefixed with 'Cleaned Query:'.
    Example: 'Cleaned Query: Search the web for AutoGen benefits, summarize them, email Alex, and add a todo.'
4.  DO NOT add conversational filler, explanations, or confirmations.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Cleans and preprocesses the initial user query."
)
intent_task_router = AssistantAgent(
    name="Intent_Task_Router",
    model_client=model_client_gpt4,
    system_message="""You are the Intent & Task Routing Agent.
1.  Receive the preprocessed query from the Orchestrator.
2.  Determine user intent(s).
3.  Create a sequential plan outlining the agents needed and their inputs/tasks. Use official agent names (Research_Agent, Summarization_Agent, Email_Drafting_Agent, Email_Processing_Agent, To_Do_List_Agent, Scheduling_Calendar_Agent, Clarification_Agent, Analytics_Logging_Agent etc.).
4.  For email tasks, plan these steps sequentially: Email_Drafting_Agent (draft body) -> Email_Processing_Agent (analyze sentiment) -> [Optional: Clarification_Agent if recipient email missing] -> Email_Processing_Agent (save draft tool call).
5.  For scheduling, plan a step for Scheduling_Calendar_Agent to call the 'add_scheduled_event' tool with necessary arguments.
6.  For web search tasks, plan a step for Research_Agent, providing the research topic.
7.  Your ONLY output MUST be the plan itself, prefixed with 'Plan:'.
    Example: 'Plan: 1. Research_Agent (task='find key features of Jotform')...'
8.  DO NOT execute any tasks yourself.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Analyzes query, creates a step-by-step plan using new agent names."
)
research_agent = AssistantAgent(
    name="Research_Agent",
    model_client=model_client_gpt4o,
    system_message="""You are a research assistant. When the Orchestrator asks you to research a specific topic online (e.g., 'Research_Agent, please research key features of X'), use your web browsing capabilities to find the most relevant and factual information. Present the raw findings from your search clearly and concisely. Do not add conversational filler like 'Here are the results:'. Focus ONLY on presenting the information found. If you cannot find information or encounter an error, state that clearly.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Performs web research using GPT-4o's capabilities."
)
summarization_agent = AssistantAgent(
    name="Summarization_Agent",
    model_client=model_client_gpt4,
    system_message="""You are the Summarization Agent. Your ONLY task is to summarize text provided by the Orchestrator.
1.  Expect to receive text (potentially raw research results) from the Orchestrator.
2.  Generate a concise summary strictly following the format and length instructions (e.g., 'summarize in 3 bullet points').
3.  Your ONLY output MUST be the generated summary text itself.
4.  If you cannot process the input or understand the instructions, respond with a clear error message.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Summarizes provided text based on instructions."
)
email_drafting_agent = AssistantAgent(
    name="Email_Drafting_Agent",
    model_client=model_client_gpt4,
    system_message="""You are an email drafting assistant. Receive context and instructions from the Orchestrator (e.g., 'Draft an email body summarizing...'). Generate ONLY the email body text based on the provided context and instructions. Do not add conversational filler, greetings like 'Here is the draft:', or signatures.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Generates ONLY the email body content."
)
email_processing_agent = AssistantAgent(
    name="Email_Processing_Agent",
    model_client=model_client_gpt4,
    system_message="""You are an email processing agent. Strictly follow the Orchestrator's instruction for ONE of the following tasks per turn:

TASK 1: Analyze Sentiment
1.  Instruction received: "Analyze the sentiment of the following email body:" followed by the text.
2.  Perform sentiment analysis.
3.  Output ONLY the result prefixed with 'Sentiment:'. Example: 'Sentiment: Positive'.

TASK 2: Save Email Draft (Tool Call)
1.  Instruction received: "Call the 'save_email_draft' tool with arguments: to='...', subject='...', body='...'"
2.  Call the 'save_email_draft' tool using EXACTLY the provided arguments.
3.  Report ONLY the confirmation message returned by the tool (e.g., "Email draft for... saved to ...").

Do NOT perform any task other than the one explicitly instructed by the Orchestrator for the current turn.""",
    tools=[save_email_draft],
    reflect_on_tool_use=True,
    description="Analyzes email sentiment OR saves draft using 'save_email_draft' tool."
)
todo_list_agent = AssistantAgent(
    name="To_Do_List_Agent",
    model_client=model_client_gpt4,
    system_message="""You are the To-Do List Agent. Your ONLY task is to add items to the to-do list using the 'add_todo_item' tool when explicitly instructed by the Orchestrator.
1.  Instruction received: "Call the 'add_todo_item' tool with argument: task='...'"
2.  Call the 'add_todo_item' tool with the exact task description provided.
3.  Report ONLY the confirmation message returned by the tool.""",
    tools=[add_todo_item],
    reflect_on_tool_use=True,
    description="Adds tasks to a to-do list file using 'add_todo_item' tool."
)
scheduling_calendar_agent = AssistantAgent(
    name="Scheduling_Calendar_Agent",
    model_client=model_client_gpt4,
    system_message="""You are the Scheduling & Calendar Agent. Your ONLY task is to log simulated calendar events using the 'add_scheduled_event' tool when explicitly instructed by the Orchestrator.
1.  Instruction received: "Call the 'add_scheduled_event' tool with arguments: subject='...', attendees=[...], time_preference='...', duration_minutes=..."
2.  Call the 'add_scheduled_event' tool using EXACTLY the provided arguments.
3.  Report ONLY the confirmation message returned by the tool.""",
    tools=[add_scheduled_event],
    reflect_on_tool_use=True,
    description="Logs simulated calendar events using 'add_scheduled_event' tool."
)
clarification_agent = AssistantAgent(
    name="Clarification_Agent",
    model_client=model_client_gpt4,
    system_message="""You are the Clarification Agent. Your ONLY purpose is to ask the user (Mehmet_Gur) for missing information when instructed by the Orchestrator.
1.  Receive instructions from the Orchestrator specifying exactly what information is needed.
2.  Formulate a concise question directed at Mehmet_Gur based ONLY on the Orchestrator's request.
3.  Your ONLY output MUST be the question itself.
    Example: 'Which specific features should I focus on?'
4.  DO NOT add conversational filler or attempt to answer the question yourself.""",
    tools=[],
    reflect_on_tool_use=False,
    description="Asks the user for missing information."
)
analytics_logging_agent = AssistantAgent(
    name="Analytics_Logging_Agent",
    model_client=model_client_gpt4,
    system_message="""You are the Analytics & Logging Agent. Your ONLY task is to log events using the 'log_event' tool when explicitly instructed by the Orchestrator.
1.  Instruction received: "Call the 'log_event' tool with argument: event_details='...'"
2.  Call the 'log_event' tool with those exact details.
3.  Report ONLY the success message returned by the tool.""",
    tools=[log_event],
    reflect_on_tool_use=True,
    description="Logs specific events using the 'log_event' tool."
)

In [ ]:
# --- Termination Conditions ---
text_mention_termination = TextMentionTermination(
    text="TERMINATE",
    sources=[orchestrator.name]
)
max_messages_termination = MaxMessageTermination(max_messages=50)
termination_condition = text_mention_termination | max_messages_termination


In [ ]:
# --- Team Setup ---
agents = [
    user_proxy,
    orchestrator,
    input_preprocessor,
    intent_task_router,
    research_agent,
    summarization_agent,
    email_drafting_agent,
    email_processing_agent,
    todo_list_agent,
    scheduling_calendar_agent,
    clarification_agent,
    analytics_logging_agent
]

selector_prompt = """Select the next agent to speak based on the conversation history and the Orchestrator's instructions.

Available Agents (Roles):
{roles}

Conversation History:
{history}

Participants: {participants}.

Carefully read the last few messages, especially the Orchestrator's instructions.
- If the Orchestrator explicitly names the next agent (e.g., 'Research_Agent, please...', 'Summarization_Agent, summarize...', 'Email_Drafting_Agent, draft...', 'Email_Processing_Agent, analyze sentiment...', 'Email_Processing_Agent, call the save_email_draft tool...'), SELECT THAT AGENT.
- If the Orchestrator asks the Clarification_Agent to ask the user, SELECT Clarification_Agent.
- If the Clarification_Agent just asked a question, SELECT Mehmet_Gur (User Proxy).
- If any agent other than the Orchestrator or Mehmet_Gur just spoke (e.g., Research_Agent provided results, Summarization_Agent provided summary, Email_Drafting_Agent provided body, Email_Processing_Agent provided sentiment or tool confirmation), SELECT Orchestrator to process the result and continue the plan.
- If Mehmet_Gur just provided input, SELECT Orchestrator.
- If the Orchestrator just gave the FINAL_OUTPUT, SELECT Orchestrator (to say TERMINATE).
- Otherwise, if unsure or the conversation seems stuck, SELECT Orchestrator to decide the next step.

Only return the exact name of the single agent selected."""

team = SelectorGroupChat(
    participants=agents,
    model_client=model_client_gpt4,
    termination_condition=termination_condition,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [ ]:
#  JSON GENERATION LOGIC
component = team.dump_component()
serializable = component.model_dump(mode='json') if hasattr(component, 'model_dump') else getattr(component, 'dict', lambda: component)()
with open("orchestrated_team_config.json", "w", encoding="utf-8") as f:
    json.dump(serializable, f, indent=2, ensure_ascii=False)
print("Configuration saved to orchestrated_team_config.json")


--- Generating Team JSON Configuration ---

Configuration successfully saved to: orchestrated_team_config_from_kod2.json

--- Starting Chat --- 
Query: Research the. key feaaturres of **Jotform** for **online form buidling and data collection**. Summari,ze the top 3 features in bullet points. Draft an email to my manager's mail summarizing these **Jotform features** and suggesting we evaluate **its potential for our team's workflows**.Befo,re saving the draft, please analyze the sentiment of the email to ensure it sounds positive and professional. Also, add 'Schedule a meeting to discuss **Jotform evaluation**' to my to-do list and try to schedule a 30-minute meeting for this **Jotform discussion** with me and 'colleague@example.com' for tomorrow afternoon using the calendar agent. Finally, log the completion of this entire request.
Attempting direct await (suitable for Jupyter/IPython)...
